In [15]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder.getOrCreate()

In [28]:
# Dataset witht all the details, produced earlier:
plip_json_input = (

    spark.read.json("gene_mapped_structures.json")

    .select("pdbStructureId", "chains", f.explode("compoundIds").alias("pdbCompoundId"))

    .select("pdbStructureId", f.explode("chains").alias("chains"))

    )

plip_json_input.show(truncate=False)

+--------------+----------------------------+
|pdbStructureId|chains                      |
+--------------+----------------------------+
|1d5m          |{A, ENSG00000227993, P01903}|
|1d5m          |{A, ENSG00000277263, P01903}|
|1d5m          |{B, ENSG00000196126, P01911}|
|1d5m          |{A, ENSG00000204287, P01903}|
|1e9b          |{A, ENSG00000168393, P23919}|
|1ere          |{F, ENSG00000091831, P03372}|
|1ere          |{C, ENSG00000091831, P03372}|
|1ere          |{D, ENSG00000091831, P03372}|
|1ere          |{A, ENSG00000091831, P03372}|
|1ere          |{E, ENSG00000091831, P03372}|
|1ere          |{B, ENSG00000091831, P03372}|
|1j3z          |{E, ENSG00000206172, P69905}|
|1j3z          |{C, ENSG00000188536, P69905}|
|1j3z          |{C, ENSG00000206172, P69905}|
|1j3z          |{G, ENSG00000188536, P69905}|
|1j3z          |{A, ENSG00000206172, P69905}|
|1j3z          |{B, ENSG00000244734, P68871}|
|1j3z          |{G, ENSG00000206172, P69905}|
|1j3z          |{D, ENSG0000024473

In [29]:
plip_json_input_v2 = (plip_json_input
                      
                      .withColumn("chainId", plip_json_input["chains.chainId"])
                      
                      .withColumn("geneId", plip_json_input["chains.geneId"])
                      
                      .withColumn("uniprotId", plip_json_input["chains.uniprot"])

                      .drop("chains")
                                            
                      )

plip_json_input_v2.show()

+--------------+-------+---------------+---------+
|pdbStructureId|chainId|         geneId|uniprotId|
+--------------+-------+---------------+---------+
|          1d5m|      A|ENSG00000227993|   P01903|
|          1d5m|      A|ENSG00000277263|   P01903|
|          1d5m|      B|ENSG00000196126|   P01911|
|          1d5m|      A|ENSG00000204287|   P01903|
|          1e9b|      A|ENSG00000168393|   P23919|
|          1ere|      F|ENSG00000091831|   P03372|
|          1ere|      C|ENSG00000091831|   P03372|
|          1ere|      D|ENSG00000091831|   P03372|
|          1ere|      A|ENSG00000091831|   P03372|
|          1ere|      E|ENSG00000091831|   P03372|
|          1ere|      B|ENSG00000091831|   P03372|
|          1j3z|      E|ENSG00000206172|   P69905|
|          1j3z|      C|ENSG00000188536|   P69905|
|          1j3z|      C|ENSG00000206172|   P69905|
|          1j3z|      G|ENSG00000188536|   P69905|
|          1j3z|      A|ENSG00000206172|   P69905|
|          1j3z|      B|ENSG000

In [31]:
# Dataset witht all the details, produced earlier:
plip_csv_output = (

    spark.read.csv("output.csv", header=True, sep=",")

    .withColumnRenamed("pdb_structure_id", "pdbStructureId")

    .withColumnRenamed("compound_id", "pdbCompoundId")
    
    .withColumnRenamed("prot_chain_id", "chainId")

    )

plip_csv_output.show()

+--------------+-------------+-------------------+-------------------+-----------------+-------+
|pdbStructureId|pdbCompoundId|   interaction_type|prot_residue_number|prot_residue_type|chainId|
+--------------+-------------+-------------------+-------------------+-----------------+-------+
|          1e9b|          ATM|         saltbridge|                 45|              ARG|      A|
|          1e9b|          ATM|         saltbridge|                 97|              ARG|      A|
|          1e9b|          ATM|              hbond|                157|              GLN|      A|
|          1e9b|          ATM|              hbond|                 76|              ARG|      A|
|          1e9b|          ATM|              hbond|                 76|              ARG|      A|
|          1e9b|          ATM|              hbond|                151|              TYR|      A|
|          1e9b|          ATM|            pistack|                 72|              PHE|      A|
|          1e9b|          ATM|

In [33]:
plip_output_target_id = (

    plip_json_input_v2

    .join(plip_csv_output, on=["pdbStructureId", "chainId"])
    
    .withColumnRenamed("interaction_type", "interactionType")

    .withColumnRenamed("prot_residue_number", "protResidueNumber")

    .withColumnRenamed("prot_residue_type", "protResidueType")

)

plip_output_target_id.show()

+--------------+-------+---------------+---------+-------------+-------------------+-----------------+---------------+
|pdbStructureId|chainId|         geneId|uniprotId|pdbCompoundId|    interactionType|protResidueNumber|protResidueType|
+--------------+-------+---------------+---------+-------------+-------------------+-----------------+---------------+
|          1e9b|      A|ENSG00000168393|   P23919|          ATM|         saltbridge|               45|            ARG|
|          1e9b|      A|ENSG00000168393|   P23919|          ATM|         saltbridge|               97|            ARG|
|          1e9b|      A|ENSG00000168393|   P23919|          ATM|              hbond|              157|            GLN|
|          1e9b|      A|ENSG00000168393|   P23919|          ATM|              hbond|               76|            ARG|
|          1e9b|      A|ENSG00000168393|   P23919|          ATM|              hbond|               76|            ARG|
|          1e9b|      A|ENSG00000168393|   P2391